In [18]:
# Install necessary libraries if not installed
# pip install PyPDF2

import re
from PyPDF2 import PdfReader

# =========================
# 1️⃣ Load PDF and extract text
# =========================
pdf_path = r"C:\Users\hamza\Desktop\Medical-Chatbot\data\pdfcoffee.com_medical-books-free-pathophysiology-of-disease-an-introduction-to-clinical-medicine-8th-edition-pdf-pdf-free.pdf"  # Replace with your PDF path
reader = PdfReader(pdf_path)
raw_text = ""

for page in reader.pages:
    raw_text += page.extract_text() + "\n"

print("Raw text length:", len(raw_text))

# =========================
# 2️⃣ Clean text
# =========================

def clean_text(text):
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Remove page numbers like "Page 12" or "12"
    text = re.sub(r'Page \d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\n\d+\n', '\n', text)  # standalone numbers on lines
    
    # Remove headers/footers if known patterns exist (e.g., book title repeated)
    text = re.sub(r'Robbins Basic Pathology.*', '', text)  # example
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

cleaned_text = clean_text(raw_text)

print("Cleaned text length:", len(cleaned_text))

# =========================
# 3️⃣ Save cleaned text to a file
# =========================
with open("cleaned_book.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print("Cleaned text saved to cleaned_book.txt")


Raw text length: 3250007
Cleaned text length: 3213634
Cleaned text saved to cleaned_book.txt


In [19]:
import langchain
print(langchain.__version__)

1.1.2


In [24]:
# =========================
# Custom text splitter for RAG
# =========================
def split_text_custom(text, chunk_size=4000, overlap=800):
    """
    Splits text into chunks of chunk_size characters with overlap.
    Approx 1000 tokens ~ 4000 characters.
    """
    chunks = []
    start = 0
    text_length = len(text)
    
    while start < text_length:
        end = min(start + chunk_size, text_length)
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap  # move start by chunk_size - overlap
    
    return chunks

# =========================
# Load cleaned text
# =========================
with open("cleaned_book.txt", "r", encoding="utf-8") as f:
    cleaned_text = f.read()

# =========================
# Split text into chunks
# =========================
chunks = split_text_custom(cleaned_text, chunk_size=4000, overlap=800)

print(f"Total chunks created: {len(chunks)}")
print("Sample chunk:\n", chunks[0][:500], "...")


Total chunks created: 1005
Sample chunk:
 Copyright © 2019 by McGraw-Hill Education. All rights reserved. Except as permitted under the United States Copyright Act of 1976, no part of this publication may be reproduced or distributed in any form or by any means, or stored in a database or retrieval system, without the prior written permission of the publisher. ISBN: 978-1-26-002651-1 MHID: 1-26-002651-5 The material in this eBook also appears in the print version of this title: ISBN: 978-1-26-002650-4, MHID: 1-26-002650-7. eBook convers ...


In [26]:
# ------------------------------
# 0️⃣ Imports (keep your existing ones)
# ------------------------------
import psycopg
from psycopg import Cursor
import ollama
from pathlib import Path

# ------------------------------
# 1️⃣ Variables
# ------------------------------
EMBED_MODEL = "embeddinggemma"  # Your Ollama embedding model
db_connection_str = "dbname=medical_rag user=postgres password=1803 host=localhost port=5432"

# If your chunks are already in memory
# chunks = [...]  # list of strings from the previous splitting step

# ------------------------------
# 2️⃣ Helper functions (reuse yours)
# ------------------------------

def calculate_embeddings(corpus: str) -> list[float]:
    response = ollama.embeddings(EMBED_MODEL, corpus)
    return response["embedding"]

def to_pgvector(vec: list[float]) -> str:
    return "[" + ",".join(str(v) for v in vec) + "]"

def save_embedding(corpus: str, embedding: list[float], cursor: Cursor) -> None:
    pg_vec = to_pgvector(embedding)
    cursor.execute(
        """
        INSERT INTO embeddings (corpus, embedding)
        VALUES (%s, %s::vector)
        """,
        (corpus, pg_vec),
    )

def similar_corpus(input_corpus: str, k: int, cursor: Cursor):
    embedding = calculate_embeddings(input_corpus)
    pg_vec = to_pgvector(embedding)

    cursor.execute(
        """
        SELECT id, corpus, embedding <=> %s::vector AS distance
        FROM embeddings
        ORDER BY distance ASC
        LIMIT %s
        """,
        (pg_vec, k),
    )

    return cursor.fetchall()

# ------------------------------
# 3️⃣ Store chunk embeddings in PostgreSQL
# ------------------------------
with psycopg.connect(db_connection_str) as conn:
    conn.autocommit = True

    with conn.cursor() as cur:
        # Drop old table if exists
        cur.execute("DROP TABLE IF EXISTS embeddings")

        # Create extension pgvector
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")

        # Create embeddings table
        cur.execute(
            """
            CREATE TABLE IF NOT EXISTS embeddings (
                id SERIAL PRIMARY KEY,
                corpus TEXT,
                embedding VECTOR(768)
            );
            """
        )

        # Iterate through your chunks
        for i, chunk in enumerate(chunks):
            emb = calculate_embeddings(chunk)
            save_embedding(chunk, emb, cur)
            if i % 50 == 0:
                print(f"Processed chunk {i+1}/{len(chunks)}")

        conn.commit()

        # Optional: test similarity search
        print("\n--- Test similarity ---")
        test_results = similar_corpus("What causes inflammation?", 3, cur)
        for r in test_results:
            print(r)


Processed chunk 1/1005
Processed chunk 51/1005
Processed chunk 101/1005
Processed chunk 151/1005
Processed chunk 201/1005
Processed chunk 251/1005
Processed chunk 301/1005
Processed chunk 351/1005
Processed chunk 401/1005
Processed chunk 451/1005
Processed chunk 501/1005
Processed chunk 551/1005
Processed chunk 601/1005
Processed chunk 651/1005
Processed chunk 701/1005
Processed chunk 751/1005
Processed chunk 801/1005
Processed chunk 851/1005
Processed chunk 901/1005
Processed chunk 951/1005
Processed chunk 1001/1005

--- Test similarity ---
(852, 'pression on local vessel endothelium to facilitate neutrophil adhesion and migration and are also potent chemoattractants for neutrophils. Neutrophils also amplify their own recruitment by releasing leukotriene LTB 4 upon urate crystal phagocytosis (see Figure 24–2 ). FIGURE 24–2 The mechanisms of the initiation and amplification of the acute inflammatory response in gout involve both cytokines and humoral mediators. The intense inflammatory

In [ ]:
import psycopg
import ollama

# ------------------------------
# Variables
# ------------------------------
EMBED_MODEL = "embeddinggemma"       # Ollama embedding model
LLM_MODEL = "llama3"                 # LLaMA 3 model for generation
db_connection_str = "dbname=rag_chatbot user=postgres password=1803 host=localhost port=5432"
TOP_K = 3                            # number of chunks to retrieve

# ------------------------------
# Helper functions
# ------------------------------

def calculate_embeddings(corpus: str) -> list[float]:
    """Get embeddings from Ollama embedding model."""
    response = ollama.embeddings(EMBED_MODEL, corpus)
    return response["embedding"]

def to_pgvector(vec: list[float]) -> str:
    """Convert list[float] to Postgres vector string."""
    return "[" + ",".join(str(v) for v in vec) + "]"

def retrieve_chunks(query: str, k: int = TOP_K):
    """Retrieve top-k most similar chunks from pgvector."""
    embedding = calculate_embeddings(query)
    pg_vec = to_pgvector(embedding)

    with psycopg.connect(db_connection_str) as conn:
        with conn.cursor() as cur:
            cur.execute(
                """
                SELECT corpus, embedding <=> %s::vector AS distance
                FROM embeddings
                ORDER BY distance ASC
                LIMIT %s
                """,
                (pg_vec, k)
            )
            results = cur.fetchall()
    
    # Return only the chunk texts
    return [r[0] for r in results]

def ask_rag(query: str) -> str:
    """Retrieve relevant chunks and generate answer using LLaMA 3."""
    # 1️⃣ Retrieve top-k chunks
    chunks = retrieve_chunks(query, TOP_K)
    context = "\n\n".join(chunks)

    # 2️⃣ Create prompt for LLaMA 3
    prompt = f"""
    You are a medical assistant. Use ONLY the following context from the textbook to answer the question.
    Context:
    {context}

    Question: {query}
    Answer:
    """

    # 3️⃣ Generate response
    response = ollama.chat(LLM_MODEL, prompt)
    return response["content"]

# ------------------------------
# Example usage
# ------------------------------
question = "What are the common causes of inflammation?"
answer = ask_rag(question)
print("\n--- RAG Answer ---")
print(answer)
